### Wstęp do Uczenia Maszynowego 
##### Laboratorium 8

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


### 1. Naive Bayes

In [16]:
X = pd.read_csv("X.csv")
y = pd.read_csv("y.csv")

### Zadanie 1
-----
Przygotuj dane w następujący sposób:

a) Ze zbioru `X` wybierz tylko zmienne `"education", "workclass", "sex", "hours-per-week"`.

b) Dokonaj transformacji dla zmiennej `"education"`

    1, education in ['Masters', 'some-college', 'Bachelors', 'Doctorate']
    0, w przeciwnym przypadku

c) Dokonaj transformacji dla zmiennej `"workclass"`

    1, workclass == 'Private'
    0, w przeciwnym przypadku

d) Dokonaj transformacji dla zmiennej `"sex"`

    1, sex == 'Male
    0, w przeciwnym przypadku

e) Dokonaj transformacji dla zmiennej `"hours-per-week"`

    1, hours-per-week <= 40 
    0, w przeciwnym przypadku

In [17]:
tmp = X[["education", "workclass", "sex", "hours-per-week"]]
tmp

,education,workclass,sex,hours-per-week
0,Bachelors,State-gov,Male,40
1,Bachelors,Self-emp-not-inc,Male,13
2,HS-grad,Private,Male,40
3,11th,Private,Male,40
4,Bachelors,Private,Female,40
...,...,...,...,...
48837,Bachelors,Private,Female,36
48838,HS-grad,NaN,Male,40
48839,Bachelors,Private,Male,50
48840,Bachelors,Private,Male,40


In [18]:
X = pd.DataFrame({"education" : tmp["education"].isin(["some-college", "Masters", "Bachelors", "Doctorate"]),
                  "workclass" : tmp["workclass"].isin(["Private"]) ,
                  "sex" : tmp["sex"].isin(["Male"]),
                  "hours" : np.where(tmp["hours-per-week"] <= 40, True, False)})

In [19]:
X

,education,workclass,sex,hours
0,True,False,True,True
1,True,False,True,True
2,False,True,True,True
3,False,True,True,True
4,True,True,False,True
...,...,...,...,...
48837,True,True,False,True
48838,False,False,True,True
48839,True,True,True,False
48840,True,True,True,True


In [20]:
y.value_counts()

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
dtype: int64

In [21]:
y.loc[y.income == '<=50K.'] = '<=50K'
y.loc[y.income == '>50K.'] = '>50K'

In [22]:
y.value_counts()

income
<=50K     37155
>50K      11687
dtype: int64

### Zadanie 2
-----
Wylicz prawdopodobieństwa dla poszczególnych zmiennych pod warunkiem `Y`. Oblicz przynależność do klasy dla pierwszej obserwacji ze zbioru testowego na postawie wyliczonych prawdopodobieństw.

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [24]:
print('<=50K:', np.mean(y_train.income == "<=50K"))
print('>50K:', np.mean(y_train.income != "<=50K"))

<=50K: 0.7597573772170041
>50K: 0.24024262278299593


In [25]:
all = pd.concat([X_train, y_train], axis=1)

In [26]:
print(round(all.groupby(['income'])['education'].value_counts(normalize = True).unstack(), 4))
print(round(all.groupby(['income'])['workclass'].value_counts(normalize = True).unstack(), 4))
print(round(all.groupby(['income'])['sex'].value_counts(normalize = True).unstack(), 4))
print(round(all.groupby(['income'])['hours'].value_counts(normalize = True).unstack(), 4))

education   False   True 
income                   
<=50K      0.8366  0.1634
>50K       0.5576  0.4424
workclass   False   True 
income                   
<=50K      0.2853  0.7147
>50K       0.3696  0.6304
sex      False   True 
income                
<=50K   0.3876  0.6124
>50K    0.1545  0.8455
hours    False   True 
income                
<=50K   0.2315  0.7685
>50K    0.4917  0.5083


In [27]:
X_test.iloc[0,:]

education    False
workclass    False
sex           True
hours        False
Name: 27660, dtype: bool

In [28]:
# dla y = "<=50K"
0.8366 * 0.2853 * 0.6124 * 0.2315 * 0.7597573772170041

0.025708736619515025

In [29]:
# dla y = ">50K"
0.5576 * 0.3696 * 0.8455 * 0.4917 * 0.24024262278299593

0.020583470833803352

Wniosek: predykcja dla obserwacji to "<=50K"

### Zadanie 3
-----
Przygotuj model Naive Bayes używająć funkcji `BernouliiNB`. 
Jakie są dostępne inne warianty modelu Naive Bayes i czym się charakteryzują?

In [30]:
from sklearn.naive_bayes import BernoulliNB

In [31]:
NB = BernoulliNB()
NB.fit(X_train, y_train.income)

BernoulliNB()

In [32]:
pred = NB.predict(X_test)

In [33]:
pred[0]
# zgadza się z wyliczeniami z Zadania 2

'<=50K'

### 2. kNN

### Zadanie 4
-----

a) Podziel losowo zbiór na część treningową i testową (7:3).

b) Przeskaluj zmienne w zbiorze treningowym i testowym za pomocą średniej i odchylenia standardowego danej zmiennej na zbiorze treningowym.

c) Dopasuj metodę k najbliższych sąsiadów dla $k = 5, 10, 20, 50, 80, 100, 150$ na zbiorze treningowym. Rozważ przynajmniej 3-krotną kroswalidację.

d) Sprawdź na zbiorze testowym współczynnik AUC dla wyliczonych $k$ z polecenia c).

e) Wybierz optymalne $k$ bazując na punkcie c) i d). Przygotuj model kNN z jednakową wagą dla wszystkich sąsiadów oraz z wagą uzależnioną od odległości między obserwacją a jej sąsiadem.

In [34]:
df = pd.read_csv("SAheart.data")

In [35]:
y = df.chd
X = df.drop(['chd'], axis = 1)
X = pd.get_dummies(X, drop_first=True)


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [37]:
from sklearn.preprocessing import StandardScaler

In [38]:
scaler = StandardScaler() 
X_train_std = scaler.fit_transform(X_train) 
X_test_std = scaler.transform(X_test)

In [39]:
import sklearn.model_selection as skm
kfold = skm.KFold(5,
                  random_state=0,
                  shuffle=True)

In [40]:
from sklearn.neighbors import KNeighborsClassifier

In [41]:
knn = KNeighborsClassifier()

In [42]:
grid = skm.GridSearchCV(knn,
                        {'n_neighbors':[5, 10, 20, 50, 80, 100, 150]}, 
                         refit=True,
                         cv=kfold,
                         scoring='roc_auc')

In [43]:
grid.fit(X_train_std, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 10, 20, 50, 80, 100, 150]},
             scoring='roc_auc')

In [44]:
grid.best_params_

{'n_neighbors': 100}

In [45]:
knn = KNeighborsClassifier(n_neighbors = grid.best_params_["n_neighbors"])
knn_distance = KNeighborsClassifier(n_neighbors = grid.best_params_["n_neighbors"], weights = "distance")

knn.fit(X_train_std, y_train)
knn_distance.fit(X_train_std, y_train)

from sklearn.metrics import roc_auc_score

print("knn: ", roc_auc_score(y_test, knn.predict_proba(X_test_std)[:,1]))
print("knn_distance :", roc_auc_score(y_test, knn_distance.predict_proba(X_test_std)[:,1]))

knn:  0.7671127687582272
knn_distance : 0.7709521720052654
